# Bases de datos

In [1]:
import pandas as pd
#import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import openpyxl
import pandas.io.formats.excel 
import time
%matplotlib inline
#pd.set_option('display.notebook_repr_html', True)
#pd.set_option('display.max_columns', 11)
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.width', 78)
pd.set_option('precision', 3)
pd.options.display.float_format = '{:.5f}'.format


In [2]:
data=pd.read_excel('annual_generation_state.xls','TOTALES',index_col=0)
data=data.loc[data['GENERATION (Megawatthours)']>0,:]

In [3]:
years=np.unique(data.index)

In [4]:
states=np.unique(data['STATE'])
states

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'US-TOTAL', 'US-Total', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype=object)

In [5]:
energies=np.unique(data['ENERGY SOURCE'])
energies

array(['Coal', 'Geothermal', 'Hydroelectric Conventional', 'Natural Gas',
       'Nuclear', 'Other', 'Other Biomass', 'Other Gases', 'Petroleum',
       'Pumped Storage', 'Solar Thermal and Photovoltaic', 'Wind',
       'Wood and Wood Derived Fuels'], dtype=object)

In [6]:
total_by_states=list(map(lambda s:list(map(lambda y:sum(data.loc[data['STATE']==states[s],'GENERATION (Megawatthours)'].loc[data.loc[data['STATE']==states[s],:].index==years[y]]),range(len(years)))),range(len(states))))

In [7]:
data_by_states=list(map(lambda e:data.loc[data['STATE']==states[e],:],range(len(states))))

In [8]:
data_by_states=list(map(lambda s:list(map(lambda y:(data.loc[data['STATE']==states[s],:].loc[data.loc[data['STATE']==states[s],:].index==years[y]]),range(len(years)))),range(len(states))))

In [9]:
#state,year
proportions_by_states=list(map(lambda s:list(map(lambda y:pd.concat([data_by_states[s][y],data_by_states[s][y]['GENERATION (Megawatthours)']/total_by_states[s][y]],axis=1),range(len(years)))),range(len(states))))

In [10]:
#state,type of energy
df=list(map(lambda s:list(map(lambda e:pd.DataFrame(np.reshape((np.column_stack(list(map(lambda y:pd.DataFrame(np.ones([1,4])*.0001) if (proportions_by_states[s][y].loc[proportions_by_states[s][y]['ENERGY SOURCE']==energies[e],:]).empty else proportions_by_states[s][y].loc[proportions_by_states[s][y]['ENERGY SOURCE']==energies[e],:],range(len(years)))))),(28,4))),range(len(energies)))),range(len(states))))

In [11]:
#state,type of energy
df_with_ret=list(map(lambda s:list(map(lambda e:pd.concat([df[s][e],(df[s][e][3]/df[s][e][3].shift(1)-1)*100],axis=1),range(len(energies)))),range(len(states))))

In [12]:
df_with_rets=list(map(lambda s:list(map(lambda e:pd.concat([df_with_ret[s][e],(df[s][e][2]/df[s][e][2].shift(1)-1)*100],axis=1),range(len(energies)))),range(len(states))))

In [77]:
df_with_rets_dates=list(map(lambda s:list(map(lambda e:df_with_rets[s][e].set_index(years),range(len(energies)))),range(len(states))))

In [80]:
doc=openpyxl.load_workbook('database.xlsx')
hoja=doc.get_sheet_by_name('Hoja1')
list(map(lambda s:list(map(lambda e:list(map(lambda f: hoja.append([""]) if pd.DataFrame.equals(df_with_rets_dates[s][e],df_with_rets_dates[0][5]) else hoja.append([df_with_rets_dates[s][e].index[f],df_with_rets_dates[s][e].iloc[f,0],df_with_rets_dates[s][e].iloc[f,1],df_with_rets_dates[s][e].iloc[f,2],df_with_rets_dates[s][e].iloc[f,3],df_with_rets_dates[s][e].iloc[f,4],df_with_rets_dates[s][e].iloc[f,5]]),range(28))),range(len(energies)))),range(len(states))))
doc.save('database.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  
